In [0]:
#!import "Storage"

In [0]:
public interface ComputeBasicCashflow : IScope<BasicCashflowDefinition>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
     builder.ForScope<ComputeBasicCashflow>(s => s
         .WithApplicability<LinearShape>(x => x.Identity.Shape == Shape.Linear)
         .WithApplicability<CompoundShape>(x => x.Identity.Shape == Shape.Compound)
    );
    //Constant is the default Case
    double[] Values => Enumerable.Repeat(Identity.InitialValue, Identity.Length).ToArray();
}

public interface LinearShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => GetLinearValues();

    private double[] GetLinearValues(){ //Should these functions be moved in the utils?
        var linear = new double[Identity.Length];
            for (var i = Identity.Shift; i < Identity.Length; i++)
                if (i == Identity.Shift)
                    linear[i] = Identity.InitialValue;
                else
                    linear[i] = linear[i-1] * Identity.Values[i - Identity.Shift];
        return linear;
    }
}

public interface CompoundShape : ComputeBasicCashflow {
    double[] ComputeBasicCashflow.Values => GetCompoundValues();

    private double[] GetCompoundValues(){
        var compound = new double[Identity.Length];
            for (var i = Identity.Shift; i < Identity.Length; i++)
                if (i == Identity.Shift)
                    compound[i] = Identity.InitialValue;
                else
                    compound[i] = compound[i-1] * Identity.Values[i - Identity.Shift];
        return compound;
    }
}

In [0]:
public interface BasicCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public FpaVariable[] rv => GetStorage().GetBasicCashflowDefinitions(Identity)
        .Select((cd,i) => new FpaVariable() {
            Layer = "Basic" + i.ToString(),
            Values = GetScope<ComputeBasicCashflow>(cd).Values,
            DataNode = Identity.DataNode,
            AmountType = Identity.AmountType,
            AccidentYear = Identity.AccidentYear,
            EstimateType = EstimateTypes.BE,
            Novelty = Novelties.C,
            AocType = AocTypes.CL
    }
    ).ToArray();
}

In [0]:
public interface BoundedCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public IEnumerable<BoundedCashflowDefinition> cd => GetStorage().GetBoundedCashflowDefinitions(Identity);

    public FpaVariable[] rv => cd.Select((x,i) => new FpaVariable() {
        Layer = "Bounded" + i.ToString(),
        Values = GenerateBoundedCashflow(x),
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray();      
}

In [0]:
public interface ReferencedCashflows : IScope<(CashflowIdentity Id, string OriginalAmountType), CashflowStorage>{

   static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<ReferencedCashflows>(s => s
            .WithApplicability<ParametrisedCashflows>(x => x.Identity.OriginalAmountType == x.Identity.Id.AmountType));

    public double[] Values => GetScope<ReferenceCashflows>(Identity.Id).rv.Select(x => x.Values).AggregateDoubleArray();
}

public interface ParametrisedCashflows : ReferencedCashflows{
    double[] ReferencedCashflows.Values => GetScope<BasicCashflows>(Identity.Id).rv.Concat(
                               GetScope<BoundedCashflows>(Identity.Id).rv)
                               .Select(x => x.Values).AggregateDoubleArray();
}

public interface WeightedReference : IScope<(CashflowIdentity Id, string ReferenceAmountType), CashflowStorage>{
    //private
    double[] referencedValues => GetScope<ReferencedCashflows>((Identity.Id with {AmountType = Identity.ReferenceAmountType}, Identity.Id.AmountType)).Values;
    double[] weights => GetStorage().GetReferencedCashflowDefinition(Identity.Id).Single(cd => cd.ReferenceAmountType == Identity.ReferenceAmountType).Values;
    
    double[] Values => weights.Zip(referencedValues, (weight, referencedValue) => weight * referencedValue).ToArray();
}

public interface ReferenceCashflows : IScope<CashflowIdentity, CashflowStorage>
{
    public FpaVariable[] rv => GetStorage().GetReferencedCashflowDefinition(Identity) //Split computation from variable creation
        .Select((cd, i) => new FpaVariable() {
        Layer = "Reference" + i.ToString(),
        Values = GetScope<WeightedReference>((Identity, cd.ReferenceAmountType)).Values,
        DataNode = Identity.DataNode,
        AmountType = Identity.AmountType,
        AccidentYear = Identity.AccidentYear,
        EstimateType = EstimateTypes.BE,
        Novelty = Novelties.C,
        AocType = AocTypes.CL
    }
    ).ToArray(); 
}